In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder



In [2]:
# Charger la DataFrame avec les colonnes spécifiques
data = pd.read_csv("dataset_rempli_specific_rep_entreprise.csv")
#data.head(74745)
data = data.loc[:, ['Problème', 'Société', "RéP-l'entreprise"]]

In [3]:
datatwo = pd.read_csv("projet 2.csv")
datatwo = datatwo.loc[:, ['Consentement du consommateur']]

C:\Users\chabc\AppData\Local\Temp\ipykernel_42532\3953872764.py:1: DtypeWarning: Columns (5,11) have mixed types. Specify dtype option on import or set low_memory=False.
  datatwo = pd.read_csv("projet 2.csv")


In [4]:
# Concaténer les colonnes
nouvelle_data = pd.concat([data, datatwo], axis=1)

In [5]:
nouvelle_data.head(78965)

,Problème,Société,RéP-l'entreprise,Consentement du consommateur
0,Managing the loan or lease,Wells Fargo & Company,Company chooses not to provide a public response,NaN
1,Using a debit or ATM card,Wells Fargo & Company,Company chooses not to provide a public response,NaN
2,"Account opening, closing, or management",Santander Bank US,Company has responded to the consumer and the ...,NaN
3,Deposits and withdrawals,Wells Fargo & Company,Company chooses not to provide a public response,NaN
4,"Loan servicing, payments, escrow account",Franklin Credit Management,Company believes it acted appropriately as aut...,NaN
...,...,...,...,...
78960,Problems caused by my funds being low,"Citizens Financial Group, Inc.",Company chooses not to provide a public response,NaN
78961,"Account opening, closing, or management",Bank of America,Company chooses not to provide a public response,NaN
78962,Getting a loan,"Navient Solutions, Inc.",Company has responded to the consumer and the ...,Consent provided
78963,Incorrect information on credit report,Equifax,Company has responded to the consumer and the ...,Consent not provided


In [6]:
# Ajouter une colonne "id_Consentement du consommateur" en fonction de la valeur de "Consentement du consommateur"
id_mapping = {'Consent not provided': 1, 'Consent provided': 2, 'Other': 3 , 'Consent withdrawn':4}  # Ajoutez les autres valeurs si nécessaire
nouvelle_data['id_Consentement du consommateur'] = nouvelle_data['Consentement du consommateur'].map(id_mapping)
nouvelle_data.head()

,Problème,Société,RéP-l'entreprise,Consentement du consommateur,id_Consentement du consommateur
0,Managing the loan or lease,Wells Fargo & Company,Company chooses not to provide a public response,NaN,NaN
1,Using a debit or ATM card,Wells Fargo & Company,Company chooses not to provide a public response,NaN,NaN
2,"Account opening, closing, or management",Santander Bank US,Company has responded to the consumer and the ...,NaN,NaN
3,Deposits and withdrawals,Wells Fargo & Company,Company chooses not to provide a public response,NaN,NaN
4,"Loan servicing, payments, escrow account",Franklin Credit Management,Company believes it acted appropriately as aut...,NaN,NaN


In [7]:
df_non_nan = nouvelle_data[nouvelle_data['Consentement du consommateur'].str.len() > 2]

In [8]:
df_non_nan.head()

,Problème,Société,RéP-l'entreprise,Consentement du consommateur,id_Consentement du consommateur
57729,Other,Capital One,Company believes it acted appropriately as aut...,Consent provided,2.0
57787,Improper contact or sharing of info,"CCS Financial Services, Inc.",Company believes the complaint is the result o...,Consent provided,2.0
57794,"Loan servicing, payments, escrow account",Bank of America,Company has responded to the consumer and the ...,Consent not provided,1.0
57799,Settlement process and costs,"Bayview Loan Servicing, LLC",Company has responded to the consumer and the ...,Consent not provided,1.0
57800,Unable to get credit report/credit score,Equifax,Company has responded to the consumer and the ...,Consent not provided,1.0


In [9]:
X = df_non_nan[['Problème', 'Société', 'RéP-l\'entreprise']]
y = df_non_nan['id_Consentement du consommateur']

In [10]:
# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
# Encoder les labels 'Yes' et 'No' en valeurs numériques
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)

In [12]:
# Effectuer l'encodage one-hot sur les colonnes catégoriques
X_train_encoded = pd.get_dummies(X_train)
X_test_encoded = pd.get_dummies(X_test)

In [13]:
# Remplacer les valeurs manquantes dans les données de test
X_test_encoded.fillna(0, inplace=True)  # Remplacer les valeurs manquantes par 0

In [14]:
# Vérifier les colonnes dans les données de test après l'encodage one-hot
missing_cols = set(X_train_encoded.columns) - set(X_test_encoded.columns)
for col in missing_cols:
    X_test_encoded[col] = 0  # Ajouter les colonnes manquantes avec des valeurs 0
X_test_encoded = X_test_encoded[X_train_encoded.columns]  # Réorganiser les colonnes


C:\Users\chabc\AppData\Local\Temp\ipykernel_42532\3694437732.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_test_encoded[col] = 0  # Ajouter les colonnes manquantes avec des valeurs 0
C:\Users\chabc\AppData\Local\Temp\ipykernel_42532\3694437732.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_test_encoded[col] = 0  # Ajouter les colonnes manquantes avec des valeurs 0
C:\Users\chabc\AppData\Local\Temp\ipykernel_42532\3694437732.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of ca

In [15]:
# Créer et entraîner le modèle RandomForestClassifier
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train_encoded, y_train_encoded)

RandomForestClassifier(random_state=42)

In [16]:
# Prédire les valeurs pour les données de test
y_pred_encoded = clf.predict(X_test_encoded)

In [17]:
# Convertir les prédictions numériques en labels originaux 'Consent not provided', 'Consent provided', etc.
y_pred = label_encoder.inverse_transform(y_pred_encoded)

In [18]:
# Évaluer la précision du modèle
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.5656121640123946


In [19]:
try:
    # Prédire les valeurs manquantes
    missing_data = nouvelle_data[nouvelle_data['id_Consentement du consommateur'].isna()]
    X_missing = missing_data[['Problème', 'Société', 'RéP-l\'entreprise']]
    predicted_values_encoded = clf.predict(X_missing)
    predicted_values = label_encoder.inverse_transform(predicted_values_encoded)
except Exception as e:
    print("Une erreur s'est produite lors de la prédiction des valeurs manquantes :", e)
    # En cas d'erreur, attribuer la valeur 3 à l'ID
    predicted_values = [3] * len(missing_data)


Une erreur s'est produite lors de la prédiction des valeurs manquantes : The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- Problème
- RéP-l'entreprise
- Société
Feature names seen at fit time, yet now missing:
- Problème_APR or interest rate
- Problème_Account opening, closing, or management
- Problème_Account terms and changes
- Problème_Adding money
- Problème_Advertising and marketing
- ...



In [20]:
# Remplacer les valeurs manquantes dans le DataFrame d'origine
nouvelle_data.loc[nouvelle_data['id_Consentement du consommateur'].isna(), 'id_Consentement du consommateur'] = predicted_values

In [21]:
nouvelle_data.isnull().any()

Problème                           False
Société                            False
RéP-l'entreprise                   False
Consentement du consommateur        True
id_Consentement du consommateur    False
dtype: bool

In [22]:
# Créer un mapping inverse
inverse_mapping = {v: k for k, v in id_mapping.items()}

# Remplacer les valeurs manquantes dans le DataFrame d'origine avec les valeurs du mapping inverse
nouvelle_data['Consentement du consommateur'] = nouvelle_data['Consentement du consommateur'].fillna(nouvelle_data['id_Consentement du consommateur'].map(inverse_mapping))

# Vérifier s'il y a des valeurs manquantes
nouvelle_data.isnull().any()


Problème                           False
Société                            False
RéP-l'entreprise                   False
Consentement du consommateur       False
id_Consentement du consommateur    False
dtype: bool

In [23]:
nouvelle_data = nouvelle_data.loc[: , ["Consentement du consommateur"]]

In [24]:
# Sauvegarder le DataFrame rempli dans un fichier Excel
nouvelle_data.to_csv("dataset_rempli_specific_Consentementduconsommateur.csv", index=False)

In [25]:
nouvelle_data.to_excel("dataset_rempli_specific_FConsentementduconsommateur.xlsx", index=False)